In [23]:
import os
os.path.abspath("./RadLex")

'/home/yuxiangliao/PhD/RadLex'

In [24]:
import os, sys
from loguru import logger

LOG_ROOT = os.path.abspath("./")
LOG_FILE = LOG_ROOT + "/logs/radlex.log"

# Remove all handlers and reset stderr
logger.remove(handler_id=None)
logger.add(
    LOG_FILE,
    level="TRACE",
    mode="w",
    backtrace=False,
    diagnose=True,
    colorize=False,
    format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
)
logger.info("\r\n" + ">" * 29 + "\r\n" + ">>> New execution started >>>" + "\r\n" + ">" * 29)
# To filter log level: TRACE=5, DEBUG=10, INFO=20, SUCCESS=25, WARNING=30, ERROR=40, CRITICAL=50
logger.add(sys.stdout, level="INFO", filter=lambda record: record["level"].no < 40, colorize=True)
logger.add(sys.stderr, level="ERROR", backtrace=False, diagnose=True, colorize=True)

9

In [25]:
with open("/home/yuxiangliao/PhD/RadLex/owlapi.xrdf", "r") as f:
  radlex = f.readlines()
    
print(len(radlex))

1499968


In [26]:
# import xml.etree.cElementTree as ET  
import xml.etree.ElementTree as ET 
tree = ET.parse("/home/yuxiangliao/PhD/RadLex/owlapi.xrdf")
root = tree.getroot()

# Analyse the content

In [72]:
child_lv1 = set()
for child in root:
    child_lv1.add(child.tag)
for nodeName in child_lv1:
    print(f"node:{nodeName}, len: {len(root.findall(nodeName))}")

node:{http://www.w3.org/2002/07/owl#}Class, len: 46761
node:{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description, len: 24076
node:{http://www.w3.org/2002/07/owl#}ObjectProperty, len: 52
node:{http://www.w3.org/2002/07/owl#}AnnotationProperty, len: 30
node:{http://www.w3.org/2002/07/owl#}NamedIndividual, len: 24075
node:{http://www.w3.org/2002/07/owl#}Ontology, len: 1
node:{http://www.w3.org/2000/01/rdf-schema#}Datatype, len: 1
node:{http://www.w3.org/2002/07/owl#}Axiom, len: 7316


In [182]:
# All the XML tags under specific path
classes = root.findall("{http://www.w3.org/2002/07/owl#}Class")

class_tags = set()
for class_node in classes:
    for ele in class_node.iter():
        class_tags.add(ele.tag)
        
for i in class_tags:
    print(i)

{http://radlex.org/RID/}JHU_White-Matter_Tractography_Atlas
{http://radlex.org/RID/}Talairach
{http://radlex.org/RID/}Preferred_name
{http://www.w3.org/2002/07/owl#}onProperty
{http://www.w3.org/2000/01/rdf-schema#}subClassOf
{http://radlex.org/RID/}Synonym_German
{http://radlex.org/RID/}JHU_DTI-81
{http://www.w3.org/2002/07/owl#}Class
{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description
{http://radlex.org/RID/}Freesurfer
{http://radlex.org/RID/}Preferred_name_German
{http://radlex.org/RID/}Acronym
{http://www.w3.org/2002/07/owl#}unionOf
{http://radlex.org/RID/}Radlex_version_of_class_change
{http://radlex.org/RID/}SNOMED_ID
{http://www.w3.org/2002/07/owl#}Restriction
{http://www.w3.org/2000/01/rdf-schema#}label
{http://radlex.org/RID/}Preferred_Name_for_Obsolete
{http://radlex.org/RID/}FMAID
{http://radlex.org/RID/}Misspelling_of_term
{http://radlex.org/RID/}Comment
{http://radlex.org/RID/}Synonym
{http://radlex.org/RID/}Replaced_by
{http://radlex.org/RID/}Definition
{http://www.w

In [61]:
idx1 = 0
idx2 = 1
node = root.find("{http://www.w3.org/2000/01/rdf-schema#}Datatype")
print(node.findall("{http://www.w3.org/2000/01/rdf-schema#}Datatype"))

[]


# Extract

In [27]:
import pandas as pd

In [80]:
OWL = "{http://www.w3.org/2002/07/owl#}"
OWL_RESTRICTION = OWL + "Restriction"
OWL_ON_PROPERTY = OWL + "onProperty"
OWL_SOME_VALUES_FROM = OWL + "someValuesFrom"
OWL_ANNOTATION_PROPERTY = OWL + "AnnotationProperty"
OWL_OBJECT_PROPERTY = OWL + "ObjectProperty"
OWL_INVERSE_OF = OWL + "inverseOf"
OWL_CLASS = OWL + "Class"

RDF = "{http://www.w3.org/1999/02/22-rdf-syntax-ns#}"
RDF_DESCRIPTION = RDF + "Description"
RDF_RESOURCE = RDF + "resource"
RDF_ABOUT = RDF + "about"

RDFS = "{http://www.w3.org/2000/01/rdf-schema#}"
RDFS_SUB_PROPERTY_OF = RDFS + "subPropertyOf"
RDFS_SUB_CLASS_OF = RDFS + "subClassOf"

RID = "{http://radlex.org/RID/}"
RID_PREFERRED_NAME = RID + "Preferred_name"

In [69]:
def hasAttrib(node:ET.Element):
    return True if node.attrib else False
    
def getAttribValue_resource(node:ET.Element):
    try:
        return node.attrib[RDF_RESOURCE]
    except KeyError:
        logger.error(f"Tag:[{node.tag}], attr:[{node.attrib}], text:[{node.text}]")
        raise
    
def getAttribValue_about(node:ET.Element):
    try:
        return node.attrib[RDF_ABOUT]
    except KeyError:
        logger.error(f"Tag:[{node.tag}], attr:[{node.attrib}], text:[{node.text}]")
        raise

## A set of relationships for concepts

In [71]:
object_properties = root.findall(OWL_OBJECT_PROPERTY)
df_relationships = pd.DataFrame(columns=["relationshipName", "subPropertyOf", "inverseOf"])

for objProp in object_properties:
    df_relationships.loc[len(df_relationships), "relationshipName"] = getAttribValue_about(objProp)
    for child in objProp:
        if child.tag == RDFS_SUB_PROPERTY_OF:
            df_relationships.loc[len(df_relationships) - 1, "subPropertyOf"] = getAttribValue_resource(child)
        if child.tag == OWL_INVERSE_OF:
            df_relationships.loc[len(df_relationships) - 1, "inverseOf"] = getAttribValue_resource(child)

print(df_relationships)


                                  relationshipName  \
0            http://radlex.org/RID/Anatomical_Site   
1                http://radlex.org/RID/Anterior_to   
2                http://radlex.org/RID/Attaches_to   
3            http://radlex.org/RID/Blood_Supply_of   
4                 http://radlex.org/RID/Bounded_by   
5                     http://radlex.org/RID/Bounds   
6                  http://radlex.org/RID/Branch_Of   
7             http://radlex.org/RID/Branch_Part_of   
8     http://radlex.org/RID/Constitutional_Part_Of   
9               http://radlex.org/RID/Contained_In   
10                  http://radlex.org/RID/Contains   
11           http://radlex.org/RID/Continuous_With   
12                 http://radlex.org/RID/Distal_to   
13               http://radlex.org/RID/Drains_Into   
14               http://radlex.org/RID/External_to   
15          http://radlex.org/RID/Has_Blood_Supply   
16                http://radlex.org/RID/Has_Branch   
17           http://radlex.o

## A set of attributes of concepts

In [75]:
annotation_properties = root.findall(OWL_ANNOTATION_PROPERTY)
df_attributes = pd.DataFrame(columns=['labelName','subPropertyOf'])

for annProp in annotation_properties:
    df_attributes.loc[len(df_attributes),'labelName'] = getAttribValue_about(annProp)
    for child in annProp:
        if child.tag==RDFS_SUB_PROPERTY_OF:
            df_attributes.loc[len(df_attributes)-1,'subPropertyOf'] = getAttribValue_resource(child)
            
print(df_attributes)

                                            labelName  \
0      http://data.bioontology.org/metadata/prefixIRI   
1                           http://radlex.org/RID/AAL   
2                       http://radlex.org/RID/Acronym   
3                     http://radlex.org/RID/CMA_Label   
4         http://radlex.org/RID/Changes_made_to_class   
5                       http://radlex.org/RID/Comment   
6                       http://radlex.org/RID/Created   
7                    http://radlex.org/RID/Definition   
8                 http://radlex.org/RID/Domain_string   
9                         http://radlex.org/RID/FMAID   
10                   http://radlex.org/RID/Freesurfer   
11                   http://radlex.org/RID/JHU_DTI-81   
12  http://radlex.org/RID/JHU_White-Matter_Tractog...   
13          http://radlex.org/RID/Misspelling_of_term   
14  http://radlex.org/RID/Preferred_Name_for_Obsolete   
15               http://radlex.org/RID/Preferred_name   
16        http://radlex.org/RID

## A set of concepts

{http://radlex.org/RID/}Preferred_name
{http://www.w3.org/2002/07/owl#}onProperty
{http://www.w3.org/2000/01/rdf-schema#}subClassOf
{http://www.w3.org/2002/07/owl#}Class
{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description
{http://radlex.org/RID/}SNOMED_ID
{http://www.w3.org/2002/07/owl#}Restriction
{http://www.w3.org/2000/01/rdf-schema#}label
{http://radlex.org/RID/}Misspelling_of_term
{http://radlex.org/RID/}Comment
{http://radlex.org/RID/}Synonym
{http://radlex.org/RID/}Replaced_by
{http://radlex.org/RID/}Definition
{http://www.w3.org/2002/07/owl#}someValuesFrom
{http://radlex.org/RID/}UMLS_ID

In [77]:
def getRelationship(node:ET.Element):
    # Relationship elements are surrounded by the "Restriction" label
    # The sub elements only contain two types: onProperty and someValuesFrom
    assert node[0].tag == OWL_RESTRICTION
    for relationship_node in node[0]:
        if relationship_node.tag == OWL_ON_PROPERTY:
            relationship_label = getAttribValue_resource(relationship_node)
        elif relationship_node.tag == OWL_SOME_VALUES_FROM:
            if hasAttrib(relationship_node):
                refers_to = [getAttribValue_resource(relationship_node)]
            else:
                refers_to = [getAttribValue_about(i) for i in relationship_node[0][0].findall(RDF_DESCRIPTION)]
                logger.trace(refers_to)
                for i in relationship_node.findall(RDF_DESCRIPTION):
                    logger.trace(i)

    return (relationship_label,refers_to)

In [82]:
classes = root.findall(OWL_CLASS)
df_classes = pd.DataFrame(columns=['classID','preferredName','definition'])
classList = []
hierarchyList = []
relationshipList = []
temp = set()

for class_node in classes:
    classItem = []
    classID = getAttribValue_about(class_node)
    classItem.append(classID)
    for ele in class_node.iter():
        # Concept Infomation
        if ele.tag == RID_PREFERRED_NAME:
            classItem.append(ele.text)
        classList.append(classItem)
        
        # Conceput hierarchy and concept relationship
        if ele.tag == RDFS_SUB_CLASS_OF:
            if ele.attrib: # Hierarchy
                borderClassID = getAttribValue_resource(ele)
                hierarchyList.append([classID,borderClassID])
            else: # Relationship
                (relationship_label,refers_to) = getRelationship(ele)
                relationshipList.append([classID, relationship_label, "|".join(refers_to)])

# The concept hierarchy table
# df_class_hierarchy = pd.DataFrame(hierarchyList, columns=['classID','subClassOf'])
# print(df_class_hierarchy)

# The concept relationship table
# df_concept_relationship = pd.DataFrame(relationshipList, columns=['conceptID','relationship','refers_to'])
# print(df_concept_relationship)

                            conceptID                        relationship  \
0       http://radlex.org/RID/RID1000     http://radlex.org/RID/Branch_Of   
1       http://radlex.org/RID/RID1000       http://radlex.org/RID/Part_Of   
2      http://radlex.org/RID/RID10000  http://radlex.org/RID/Contained_In   
3      http://radlex.org/RID/RID10001  http://radlex.org/RID/Contained_In   
4      http://radlex.org/RID/RID10001      http://radlex.org/RID/Has_Part   
...                               ...                                 ...   
82115   http://radlex.org/RID/RID9996       http://radlex.org/RID/Part_Of   
82116   http://radlex.org/RID/RID9997       http://radlex.org/RID/Part_Of   
82117   http://radlex.org/RID/RID9998     http://radlex.org/RID/Member_Of   
82118   http://radlex.org/RID/RID9998       http://radlex.org/RID/Part_Of   
82119   http://radlex.org/RID/RID9999  http://radlex.org/RID/Contained_In   

                            refers_to  
0        http://radlex.org/RID/RID9